## Введение в обработку естественного языка

Домашнее задание №7

Урок 7. Сверточные нейронные сети для анализа текста

*Формат именования файла домашней работы: FIO_NLP_HW_N.ipynb, где N - номер домашнего задания*

**

### Задание
Берем отызывы за лето (из архива с материалами или предыдущего занятия)

1. Учим conv сеть для классификации
2. Рассмотреть 2-а варианта сеточек

2.1 Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/

2.2 Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)

Сравнить две архитектуры с предобученными весами и когда tf.keras.layers.Embedding обучается сразу со всей сеточкой, что получилось лучше

**Установка библиотек**

In [1]:
!pip install pymorphy2
!pip install stop_words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 89.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13721 sha256=3b33c7547083711cf8441575b451794c22ae759827f55577b099dd19a40ade39
  Stored in directory: /root/.cache/pip/wheels/70/4a/46/1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built docopt
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for stop_words: filename=stop_words-2018.7.23-py3-none-any.whl size=32910 sha256=e7adc819ed496edef00426a4ee9bbc2613e79db8b534bf95c446c6f7b45a943f
  Stored in directory: /root/.cache/pip/wheels/da/d8/66/395317506a23a9d1d7de433ad6a7d9e6e16aab48cf028a0f60
Succe

In [2]:
import pandas as pd
import numpy as np
import re

In [4]:
df = pd.read_excel('отзывы за лето (1).xls')
df.head(5)

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [5]:
df.isna().sum()

Rating     0
Content    3
Date       0
dtype: int64

In [6]:
df = df.dropna()

In [7]:
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from string import punctuation

exclude = set(punctuation)
sw = set(get_stop_words("ru"))
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in exclude]
    return " ".join(txt)

df['text'] = df['Content'].apply(preprocess_text)
# df = df[df['Rating'] != 3]
# df['target'] = (df['Rating'] > 3)*1

In [8]:
df.Rating.value_counts()

5    14584
1     2276
4     2137
3      911
2      748
Name: Rating, dtype: int64

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.text, df.Rating, test_size=0.3, random_state=42, stratify=df.Rating)

In [10]:
train_corpus = " ".join(X_train)
train_corpus = train_corpus.lower()

In [11]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens = word_tokenize(train_corpus)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Отфильтруем данные и соберём в корпус N наиболее частых токенов

In [12]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [13]:
max_words = 10000
max_len = 100
num_classes = 6

# Training
epochs = 5
batch_size = 512
print_batch_n = 100

In [14]:
from nltk.probability import FreqDist

dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [15]:
tokens_filtered_top[:10]

['приложение', 'всё', 'и', 'очень', 'удобно', 'в', 'я', 'на', 'работать', 'с']

In [16]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [17]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(str(text).lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [18]:
X_train = np.asarray([text_to_sequence(text, max_len) for text in X_train], dtype=np.int32)
X_test = np.asarray([text_to_sequence(text, max_len) for text in X_test], dtype=np.int32)

In [19]:
X_train

array([[   0,    0,    0, ...,    2,  166,  171],
       [   0,    0,    0, ...,   12, 3752, 2665],
       [   0,    0,    0, ...,    0,    0,    5],
       ...,
       [   0,    0,    0, ...,    0,    0,   28],
       [   0,    0,    0, ...,    4,    5,  180],
       [   0,    0,    0, ...,    5, 2521,    1]], dtype=int32)

## Построение сетей

In [20]:
import keras
from keras.utils import np_utils

y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)

In [21]:
y_train

array([[0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.],
       ...,
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.]], dtype=float32)

In [24]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.callbacks import TensorBoard
# from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping

### tf.keras.layers.Embedding по умолчанию

In [25]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [26]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [27]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    # callbacks=[tensorboard, early_stopping],
                    )

Epoch 1/5
26/26 [==============================] - 17s 259ms/step - loss: 1.3688 - accuracy: 0.6616 - val_loss: 1.0124 - val_accuracy: 0.7026
Epoch 2/5
26/26 [==============================] - 6s 219ms/step - loss: 0.9539 - accuracy: 0.7064 - val_loss: 0.8603 - val_accuracy: 0.7026
Epoch 3/5
26/26 [==============================] - 6s 231ms/step - loss: 0.7795 - accuracy: 0.7064 - val_loss: 0.7379 - val_accuracy: 0.7026
Epoch 4/5
26/26 [==============================] - 6s 220ms/step - loss: 0.7011 - accuracy: 0.7104 - val_loss: 0.6906 - val_accuracy: 0.7490
Epoch 5/5
26/26 [==============================] - 5s 188ms/step - loss: 0.6488 - accuracy: 0.7694 - val_loss: 0.6643 - val_accuracy: 0.7718


In [28]:
preds = model.predict(X_test)

194/194 [==============================] - 0s 2ms/step


In [29]:
preds = [np.argmax(num) for num in preds]

In [30]:
from sklearn.metrics import classification_report

print(classification_report([np.argmax(l) for l in y_test], preds))

              precision    recall  f1-score   support

           1       0.43      0.77      0.55       683
           2       0.00      0.00      0.00       225
           3       0.00      0.00      0.00       273
           4       0.00      0.00      0.00       641
           5       0.85      0.97      0.91      4375

    accuracy                           0.77      6197
   macro avg       0.26      0.35      0.29      6197
weighted avg       0.65      0.77      0.70      6197



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Модель хорошо предсказывает 5 класс и очень плохо остальные.

### tf.keras.layers.Embedding предобученными векторами

In [31]:
!pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [32]:
from gensim.models import KeyedVectors
import gensim

In [33]:
# taiga_upos_skipgram_300_2_2018

!wget https://rusvectores.org/static/models/rusvectores4/taiga/taiga_upos_skipgram_300_2_2018.vec.gz

!gunzip taiga_upos_skipgram_300_2_2018.vec.gz

--2023-04-20 19:08:30--  https://rusvectores.org/static/models/rusvectores4/taiga/taiga_upos_skipgram_300_2_2018.vec.gz
Resolving rusvectores.org (rusvectores.org)... 172.104.228.108
Connecting to rusvectores.org (rusvectores.org)|172.104.228.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 346385366 (330M) [application/x-gzip]
Saving to: ‘taiga_upos_skipgram_300_2_2018.vec.gz’

taiga_upos_skipgram 100%[===================>] 330.34M  14.0MB/s    in 25s     

2023-04-20 19:08:57 (13.0 MB/s) - ‘taiga_upos_skipgram_300_2_2018.vec.gz’ saved [346385366/346385366]



In [34]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['text'])
vocab_size = len(tokenizer.word_index) + 1

In [35]:
embeddings_index = {}
with open('taiga_upos_skipgram_300_2_2018.vec') as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        word = word.split('_')[0]
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 209666 word vectors.


In [36]:
embedding_dim = 300
hits = 0
missed = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        missed += 1

print("Converted %d words (%d missed)" % (hits, missed))

Converted 5672 words (7961 missed)


In [37]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=300, input_length=max_len, weights=[embedding_matrix],
    trainable=False))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [38]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [39]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    # callbacks=[tensorboard, early_stopping],
                    )

Epoch 1/5
26/26 [==============================] - 2s 27ms/step - loss: 1.6058 - accuracy: 0.1396 - val_loss: 1.4589 - val_accuracy: 0.3831
Epoch 2/5
26/26 [==============================] - 0s 14ms/step - loss: 1.2903 - accuracy: 0.6596 - val_loss: 1.0844 - val_accuracy: 0.7317
Epoch 3/5
26/26 [==============================] - 0s 14ms/step - loss: 0.9583 - accuracy: 0.7399 - val_loss: 0.8711 - val_accuracy: 0.7407
Epoch 4/5
26/26 [==============================] - 0s 14ms/step - loss: 0.8521 - accuracy: 0.7492 - val_loss: 0.8376 - val_accuracy: 0.7407
Epoch 5/5
26/26 [==============================] - 0s 14ms/step - loss: 0.8117 - accuracy: 0.7502 - val_loss: 0.7972 - val_accuracy: 0.7600


In [40]:
preds = model.predict(X_test)
preds = [np.argmax(num) for num in preds]
print(classification_report([np.argmax(l) for l in y_test], preds))

194/194 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           1       0.42      0.65      0.51       683
           2       0.00      0.00      0.00       225
           3       0.00      0.00      0.00       273
           4       0.00      0.00      0.00       641
           5       0.82      0.96      0.89      4375

    accuracy                           0.75      6197
   macro avg       0.25      0.32      0.28      6197
weighted avg       0.63      0.75      0.68      6197



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Качество предсказаний модели чуть снизилось при использовании предобученных векторов.